# Action PixelBytes: Catching Insights in Unified Multimodal Sequences

## Description

**Action-PixelBytes** est un modèle conçu pour générer simultanément du texte, des images, des animations pixel par pixel et des actions-états sous forme de séquences. L'objectif de ce projet est d'explorer un embedding unifié qui permet une génération multimodale cohérente, facilitant ainsi l'interaction entre différentes formes de données.

## Dataset

Pour ce projet, nous utilisons le dataset **PixelBytes-PokemonSprites**. Contrairement à la version précédente, **PixelBytes-Pokemon**, cette version est structurée à la volée pour un format unifié. Cela signifie que les données sont préparées de manière à être directement utilisées pour l'entraînement du modèle, ce qui simplifie le processus d'embedding.

## Tokenizer

Le tokenizer joue un rôle dans la préparation des données pour le modèle. Voici comment il fonctionne pour chaque type de donnée :

### Traitement du Texte
Le texte est d'abord normalisé et converti en minuscules. Ensuite, il est encodé en format ASCII pour garantir que tous les caractères sont traités de manière uniforme. Cette étape permet de simplifier le texte avant de le transformer en une séquence de tokens.

### Traitement des Images
Les images, y compris les GIFs, sont traitées en plusieurs étapes. Chaque image est convertie en un espace colorimétrique LAB, qui est plus adapté à certaines analyses d'image. Ensuite, chaque frame d'une image animée est quantifiée selon une palette de couleurs prédéfinie, ce qui permet de réduire la complexité des données tout en préservant les informations essentielles.

### Traitement des Actions-États
Les actions-états sont normalisées pour assurer que toutes les valeurs sont sur la même échelle. Cela facilite la comparaison et l'analyse des états d'action. Les états sont ensuite quantifiés selon un ensemble prédéfini d'états d'action, ce qui permet au modèle de mieux comprendre les relations entre différentes actions.

### Création de Séquences
Les séquences sont créées en utilisant un contexte spatial et temporel. Cela signifie que pour chaque séquence, le modèle prend en compte non seulement l'entrée actuelle, mais aussi les entrées précédentes. Cela permet de générer des entrées de plusieurs éléments qui contiennent des informations pertinentes pour la tâche à accomplir.

## État du Projet

Le code est encore en cours de développement. Bien que les principales fonctionnalités du tokenizer et du traitement des données soient implémentées, des améliorations et des optimisations sont à venir. L'objectif est de rendre le modèle plus robuste et efficace pour la génération multimodale.

## Prochaines Étapes

- Finaliser l'architecture du modèle.
- Implémenter l'entraînement et l'évaluation.
- Optimiser les performances du modèle.
- Effectuer des tests approfondis sur différents types de données multimodales.


In [1]:
!pip install -q git+https://github.com/fabienfrfr/PixelBytes.git@main

In [2]:
# only in kaggle for HF
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
# no warning msg during train
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# our approach
from pixelbytes import *

[INFO] Official mamba_ssm CUDA modules not found. Falling back to an unofficial implementation. Performance may be affected.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler

def count_parameters_in_k(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000

In [4]:
tokenizer = ActionPixelBytesTokenizer()

anim_dataset = load_dataset("ffurfaro/PixelBytes-PokemonSprites")

KeyError: 'safe'

In [5]:
# Paramètres
VOCAB_SIZE = tokenizer.vocab_size
EMBED_SIZE = 128
HIDDEN_SIZE = 512
NUM_LAYERS = 2
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ACCUMULATION_STEPS = 4
SEQ_LENGTH = 1024
STRIDE = 512

In [6]:
# Initialisation du modèle
config = ModelConfig_(vocab_size=VOCAB_SIZE, embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS)
model = BestPreTrainedModel(config).to(DEVICE)
print(f"Le modèle a {count_parameters_in_k(model):.2f}k paramètres entraînables.")

# Préparation des données
dataset = TokenizedDataset(anim_dataset['train'], tokenizer, SEQ_LENGTH, STRIDE)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=-100)
scaler = GradScaler() if torch.cuda.is_available() else None

Le modèle a 4806.27k paramètres entraînables.


NameError: name 'anim_dataset' is not defined

In [7]:
# Entraînement
model.train_model(dataloader, optimizer, criterion, DEVICE, scaler, EPOCHS, ACCUMULATION_STEPS)

# Sauvegarde du modèle
model.save_pretrained('lstm_pokemon_sprite_model')

# Test de génération
test_input = next(iter(dataloader))['input_ids'][:1].to(DEVICE)
generated = model.generate(test_input, max_length=100)
print("Generated sequence:", generated)

NameError: name 'dataloader' is not defined